In [12]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import platform
import pickle

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!rm -r DLAV-2022
!git clone https://github.com/vita-epfl/DLAV-2022.git
path = os.getcwd() + '/DLAV-2022/homeworks/hw2/test_batch'

Cloning into 'DLAV-2022'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 83 (delta 31), reused 60 (delta 16), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [15]:
# Set the variable to the location of the trained model
model_path = "drive/MyDrive/Colab Notebooks/CNN_pytorch.ckpt"

In [16]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        
        self.feature_extractor = torch.nn.Sequential(
            
          torch.nn.Conv2d(3, 20, kernel_size=(5,5)),
          torch.nn.BatchNorm2d(20),
          torch.nn.ReLU(inplace=True),
          torch.nn.MaxPool2d(kernel_size=(2,2), stride=2),

          torch.nn.Conv2d(20, 20, kernel_size=(4,4)),
          torch.nn.BatchNorm2d(20),
          torch.nn.ReLU(inplace=True),
          torch.nn.MaxPool2d(kernel_size=(2,2))

        )

        self.classifier = torch.nn.Sequential(
            
          torch.nn.Linear(20 * 5 * 5, 10)

        )

        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Tensor `x` with `x.view`.                                                    #
        ################################################################################
        
        x = self.feature_extractor(x)
        # print(x.size())
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.classifier(x)

        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [24]:
def predict_usingCNN(X):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    # Note: For the predictions, you have to return the index of the max    #
    # value                                                                 #
    #########################################################################
  
    net = ConvNet()
    checkpoint = torch.load("drive/MyDrive/Colab Notebooks/CNN_pytorch.ckpt")
    net.load_state_dict(checkpoint)
    y_pred = net(X)
    # y_pred = torch.argmax(net(X), dim=1)
    print(y_pred.size())

    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred
   

In [18]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

In [25]:
# Data Manipulation
mean_pytorch = np.array([0.4914, 0.4822, 0.4465])
std_pytorch = np.array([0.2023, 0.1994, 0.2010])
X_pytorch = np.divide(np.subtract( X/255 , mean_pytorch[np.newaxis, :,np.newaxis,np.newaxis]), std_pytorch[np.newaxis, :,np.newaxis,np.newaxis])

# Run Prediction and Evaluation
prediction_cnn = predict_usingCNN(torch.from_numpy(X_pytorch).float())
acc_cnn = np.sum(prediction_cnn == Y)/len(X_pytorch)
print("CNN Accuracy= %f"%(acc_cnn))

torch.Size([10000, 20, 5, 5])
torch.Size([10000, 10])
CNN Accuracy= 0.000000
